## Preparation

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader
import numpy as np

In [ ]:
# device check(CFDLab)
device=torch.device('cuda:1') #先調1再調0
print(torch.cuda.is_available())
x=torch.randn(100).to(device) # Gives warning

In [ ]:
device=torch.device('cuda:0') # Fix warning
x=torch.randn(100).to(device) # No warning

In [ ]:
# device check(my pc)
device=torch.device('cuda')
print(torch.cuda.is_available())
x=torch.randn(100).to(device) # No warning

## Parameters

In [ ]:
batch=100 #batchsize
opt=1 # optimizer
tol=1e-4 # tolerance
n=[6400, 1000, 1000, 1000, 1000, 1000, 6400] #model layout
PATH= "ABS_SUP_FCN_TEST" # model name, saved as..

## Data

In [ ]:
# Step 1. Prepare data-1/2
# DataLoader wraps a Dataset and provides minibatches, shuffling, multithreading, for you
data_in=np.loadtxt('./data/preserved/input_div_U_2_1s.dat')
data_out=np.loadtxt('./data/preserved/output_P_1s.dat')
x_in=torch.Tensor(data_in)
y_in=torch.Tensor(data_out)
x_in=x_in.to(device)
y_in=y_in.to(device)

In [ ]:
x=x_in
y=y_in
print(x.size())
print(y.size())

In [ ]:
loader=DataLoader(TensorDataset(x, y), batch_size=batch)

## Model

In [ ]:
# Step 2. Create model 建立model習慣建立class
class TwoLayerNet(torch.nn.Module):
    def __init__(self, D_in, A, B, C, D, E, D_out):
        super(TwoLayerNet, self).__init__()
        self.linear_1=torch.nn.Linear(D_in, A)
        self.linear_2=torch.nn.Linear(A, B)
        self.linear_4=torch.nn.Linear(B, C)
        self.linear_5=torch.nn.Linear(C, D)
        self.linear_6=torch.nn.Linear(D, E)
        self.linear_3=torch.nn.Linear(E, D_out)
    
    # Step 3. Forward pass-1/2    # Step 4. Backward pass-1/2
    def forward(self, x):
        a=self.linear_1(x)
        a_relu=torch.nn.functional.relu(a) #為何activation and hidden layer 的實現方式不同
        b=self.linear_2(a_relu) 
        b_relu=torch.nn.functional.relu(b)
        c=self.linear_4(b_relu) 
        c_relu=torch.nn.functional.relu(c)
        d=self.linear_5(c_relu) 
        d_relu=torch.nn.functional.relu(d)
        e=self.linear_6(d_relu) 
        e_relu=torch.nn.functional.relu(e)
        y_pred=self.linear_3(e_relu) 
        return y_pred
    
model= TwoLayerNet(D_in=6400, A=n[1], B=n[2], C=n[3], D=n[4], E=n[5], D_out=6400)
model=model.to(device) #這行是什麼意思? A:将模型加载到相应的设备

## Training

In [ ]:
if opt==0:
    optimizer=torch.optim.RMSprop(model.parameters(), lr=0.001, alpha=0.99, eps=1e-08, weight_decay=0, momentum=0, centered=False)
elif opt==1:
    optimizer=torch.optim.Adam(model.parameters(), lr=1e-3)


scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5000, gamma=0.1)

In [ ]:
loss_epoch=[]
loss_values = []
loss=1
epochs=0

print("Epochs    Loss")

while(loss>tol):
    epochs=epochs+1
    scheduler.step()
    
    for x_batch, y_batch in loader:
        # Forward pass
        y_pred=model(x_batch)        
        loss=torch.nn.functional.mse_loss(y_pred, x_batch)
        
        # Backward pass
        loss.requires_grad_(True)
        loss.backward()
        
        # Update Weights
        optimizer.step()
        optimizer.zero_grad()
        #print("1 batch")
    
    loss_epoch.append(epochs)
    loss_values.append(loss.item())
    
    if epochs%1==0:
        print("Epochs: ", epochs, "; Loss: ", loss.item())
        
    loss=loss.item()

print(epochs, "    ", loss.item())

#Plot loss function
from matplotlib import pyplot as plt
plt.plot(loss_epoch, loss_values)
plt.xlabel('epochs')
plt.ylabel('loss')

## Result

In [ ]:
# Append name with time
from datetime import datetime
ct=datetime.now()
ctime="_"+str(ct.year)+str(ct.month)+str(ct.day)+"_"+str(ct.hour)+str(ct.minute)+str(ct.second)
PATH="./result/"+PATH+ctime+".pth"

In [ ]:
#Plot loss function
from matplotlib import pyplot as plt
plt.plot(loss_epoch, loss_values)
plt.xlabel('epochs')
plt.ylabel('loss')
plt.semilogy()
plt.savefig(PATH+'.png')

## Export

In [ ]:
device=torch.device('cpu')
model=model.to(device)
traced_net=torch.jit.trace(model, (torch.randn(1,6400)).to(device))
traced_net.to(torch.float64)
torch.jit.save(traced_net, PATH)